In [1]:
import torch
import torch.utils.data as data
from models import Model
import matplotlib.pyplot as plt
import const
from dataset import DrivingDataset
import cv2
import numpy as np
import loss_utils
from scipy.misc import imresize

/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [3]:
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline

from vis.utils import utils

# load model weights
model = Model(const.CURR_MODEL)
print('Loading weights: {}'.format(const.MODEL_WEIGHTS))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.load_state_dict(torch.load(const.MODEL_WEIGHTS, map_location=device))
model.eval()

# obtain inputs and labels
BATCH_SIZE = 1
NUM_SHUFFLES = 3
train_dataset = DrivingDataset(const.TRAIN_DRIVING_LOG_PATH)
train_dataloader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)
inputs, measurements, labels, high_level_controls = next(iter(train_dataloader))
for i in range(NUM_SHUFFLES):
    inputs, measurements, labels, high_level_controls = next(iter(train_dataloader))


# Display original pictures
#plt.figure(1), plt.imshow(cv2.cvtColor(np.asarray(inputs[0]).astype('uint8'), code=cv2.COLOR_BGR2RGB))

# Make the input require a gradient
inputs.requires_grad_()

inputs = inputs.to(device)
labels = labels.to(device)
measurements = measurements.to(device)

outputs = model(inputs, measurements)
pred = outputs[high_level_controls][0][0] # steer prediction
print('Predicted {}'.format(pred))

ImportError: No module named 'vis.utils'